In [1]:
class World:
    
    
    def __init__(self, length, width):
        self.length = length
        self.width = width
        self.creatures = []
        self.food = []
        self.map = [[" " for i in range(length)] for j in range(width)]
        
    
    def set_food(self, Food):
        self.map[Food.location[0]][Food.location[1]] = "X" 
        self.food.append(Food)
        self.update()
    
    
    def food_eaten(self, creature_location):
        for i in self.food:
            if creature_location == i.location:
                food_value = i.value
                self.food.remove(i)
                self.update()
                return food_value
        self.update()
    
    
    def get_map_piece(self, length, width, center):
        map_piece = [[" " for i in range(length)] for j in range(width)]
        for i in range(length):
            for j in range(width):
                if((center[0]-int(length/2))+i > self.length-1 or (center[1]-int(width/2))+j > self.width-1):
                    map_piece[i][j] = "|"
                elif((center[0]-int(length/2))+i < 0 or (center[1]-int(width/2))+j < 0):
                    map_piece[i][j] = "|"
                else:
                     map_piece[i][j] = self.map[(center[0]-int(length/2))+i][(center[1]-int(width/2))+j]
        return map_piece
        
    
    def print_map(self):
        #for i in range(self.length):
            #print("_", end=" ")
        for i in range(self.length):
            print("|", end=" ")
            for j in range(self.width):
                print(self.map[j][i], end=" ")
            print("|")
        #for i in range(self.length):
            #print("_", end=" ")
    
    
    def add_creature(self, creature):
        self.creatures.append(creature)
        self.update()
    
    
    
    def integrate(self, time):
        for i in range(time):
            for j in self.creatures:
                j.update_vision()
                j.live()
            self.update()
        
    
    def update(self):
        self.map = [[" " for i in range(self.length)] for j in range(self.width)]
        for j in self.food:
            self.map[j.location[0]][j.location[1]] = "X"
        for i in self.creatures:
            if self.map[i.location[0]][i.location[1]] == " ":
                self.map[i.location[0]][i.location[1]] = "O"
            else:
                self.map[i.location[0]][i.location[1]] = "*"
        self.print_map()
        print()


In [2]:
class Food:
    def __init__(self, x, y):
        self.value = 20
        self.uses = 2
        self.location = [x, y]

In [3]:
import random

class Creature:
    size = [1,1]
   
    
    # Initialize creature class
    def __init__(self, x=0, y=0, name="creature", health=100, food=100):
        self.location = [x, y]
        self.vision = []
        self.food = food
        self.name = name
        self.health = health
        self.food_spots = []
        self.vision_strength = 2
        self.update_vision()
    
    def status(self):
        print(self.name)
        print("Health: ", self.health)
        print("Food: ", self.food)
        print("Location: ", self.location)
        print("Line of Sight: ")
        for i in range(len(self.vision)):
            for j in range(len(self.vision[0])):
                print(self.vision[j][i], end=" ")
            print()
    
    
    def live(self):
        self.find_food()
    
    # find_food moves creature to food
    def find_food(self):
        if self.food_spots:
            self.go_to_food()
        #elif self.vision[2][2] == "*":
         #   self.eat()
        else:
            direction = random.randint(0,1)
            step = random.randint(0,1)
            move_possible = self.viable_move(step, direction)
            while(move_possible == False):
                direction = random.randint(0,1)
                step = random.randint(0,1)
                move_possible = self.viable_move(step, direction)
            if step == 0:
                self.move(-1,direction)
            else:
                self.move(1,direction)
            self.update_vision()
            
    
    # update_vision updates the sight line of the creature 
    def update_vision(self):
        self.vision = world.get_map_piece(self.vision_strength * 2 + 1, self.vision_strength*2 + 1, self.location)
        for i in range(self.vision_strength * 2 + 1):
            for j in range(self.vision_strength*2 + 1):
                if (self.vision[i][j] == "X" or self.vision[i][j] == "*") and ([i-self.walls_in_sight([i,j],'y'),j-self.walls_in_sight([i,j],'x')] not in self.food_spots):
                    #print("before: ", i,j, " ", self.vision[i][j])
                    #print([i-self.walls_in_sight([i,j],'y'),j-self.walls_in_sight([i,j],'x')])
                    self.food_spots.append([i-self.walls_in_sight([i,j],'y'),j-self.walls_in_sight([i,j],'x')])
                    
                
    # no_food_in_sight checks if there is food in the current sight line
    def no_food_in_sight(self):
        for i in range(len(self.vision)):
            for j in range(len(self.vision[0])):
                if self.vision[i][j] == "X" or self.vision[i][j] == "*":
                    return False
        return True
       
        
    def move(self, step, direction):
        if direction == 0:
            self.location[0] += step
        elif direction == 1:
            self.location[1] += step
    
    def viable_move(self, step, direction):
        if direction == 0:
            if self.location[0] + step >= world.width or self.location[0] + step <= 0:
                return False
        elif direction == 1:
            if self.location[1] + step >= world.length or self.location[1] + step <= 0:
                return False
        else:
            return True
        
    
    def walls_in_sight(self, cord, x_or_y):
        wall = 0
        if x_or_y == 'x':
            for i in range(len(self.vision)):
                if self.vision[cord[0]][i] == "|":
                    wall += 1
            if self.location[1] > self.vision_strength:
                wall += -(self.location[1] - self.vision_strength)
        else:
            for i in range(len(self.vision)):
                if self.vision[i][cord[1]] == "|":
                    wall += 1
            if self.location[0] > self.vision_strength:
                wall += -(self.location[0] - self.vision_strength)
        return wall
    
    def move_towards_food(self, move_time, fastest_time):
        if self.location != move_time[fastest_time]:
            not_done = True
            x_or_y = random.randint(0,1)
            if x_or_y == 0 and self.location[0] != move_time[fastest_time][0]:
                if self.location[0] < move_time[fastest_time][0]:
                    self.move(1, 0)
                elif self.location[0] > move_time[fastest_time][0]:
                    self.move(-1, 0)
                not_done = False
            elif self.location[1] != move_time[fastest_time][1]: 
                if self.location[1] < move_time[fastest_time][1]:
                    self.move(1, 1)
                elif self.location[1] > move_time[fastest_time][1]:
                    self.move(-1, 1)
                not_done = False
            if(not_done):
                if x_or_y == 1 and self.location[1] != move_time[fastest_time][1]:
                    if self.location[1] < move_time[fastest_time][1]:
                        self.move(1, 1)
                    elif self.location[1] > move_time[fastest_time][1]:
                        self.move(-1, 1)
                elif self.location[0] != move_time[fastest_time][0]:
                    if self.location[0] < move_time[fastest_time][0]:
                        self.move(1, 0)
                    elif self.location[0] > move_time[fastest_time][0]:
                        self.move(-1, 0)
        else:
            self.eat()
    
    # moves creature on top of the food     
    def go_to_food(self):
        move_time = {}
        for i in self.food_spots:
            time = abs(i[0] - self.location[0]) + abs(i[1]- self.location[1])
            move_time[time] = i
        fastest_time = min(move_time)
        self.move_towards_food(move_time, fastest_time)

    
    # creature eats food
    def eat(self):
        on_food = False
        for i in self.food_spots:   
            if self.location == i:
                on_food = True
        if on_food:
            self.food += world.food_eaten(self.location)
            self.food_spots.remove(self.location)
        print(self.food_spots)
        self.update_vision()
            
            

In [4]:
world = World(10, 10)
global world

food1 = Food(1, 4)
food2 = Food(8, 9)
food3 = Food(4, 7)
world.set_food(food1)
world.set_food(food2)
world.set_food(food3)
creature = Creature(2, 8, "Creature 1")
creature2 = Creature(1,3, "Creature 2")
world.add_creature(creature)
world.add_creature(creature2)


|                     |
|                     |
|                     |
|                     |
|   X                 |
|                     |
|                     |
|                     |
|                     |
|                     |

|                     |
|                     |
|                     |
|                     |
|   X                 |
|                     |
|                     |
|                     |
|                     |
|                 X   |

|                     |
|                     |
|                     |
|                     |
|   X                 |
|                     |
|                     |
|         X           |
|                     |
|                 X   |

|                     |
|                     |
|                     |
|                     |
|   X                 |
|                     |
|                     |
|         X           |
|     O               |
|                 X   |

|                     |
|           

In [5]:

print(creature.food_spots)
creature.status()
print(creature2.food_spots)
creature2.status()

[[4, 6]]
Creature 1
Health:  100
Food:  100
Location:  [2, 8]
Line of Sight: 
          
        X 
          
          
| | | | | 
[[1, 4]]
Creature 2
Health:  100
Food:  100
Location:  [1, 3]
Line of Sight: 
|         
|         
|         
|   X     
|         


In [6]:
world.integrate(10)

|                     |
|                     |
|                     |
|                     |
|   *                 |
|                     |
|                     |
|     O   X           |
|                     |
|                 X   |

|                     |
|                     |
|                     |
|                     |
|   O                 |
|                     |
|                     |
|       O X           |
|                     |
|                 X   |

[]
|                     |
|                     |
|                     |
|                     |
|   O                 |
|                     |
|                     |
|       O X           |
|                     |
|                 X   |

|                     |
|                     |
|                     |
|                     |
|     O               |
|                     |
|                     |
|         *           |
|                     |
|                 X   |

|                     |
|        

TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'